In [4]:
# Loading Beat Annotation Data from oma.digital
# import omapy library
from omapy import oma
import pandas as pd

# specify the oma cloud you are using
url = "https://oma.digital/"
# specify your security token from your web account
token = "uvmg82kmatt4hruacmi39d47hhvnpkp5"

# oma provides a handle to omapy API features via Login()
oma = oma.Login(url, token)

recording_ids = [189068, 189072, 189096, 189114, 189123, 189518, 189141, 189147]

recordings = []
for recording_id in recording_ids:
    recordings.append(oma.recording(recording_id))

recordings


,0
0,<omapy.oma.Recording object at 0x7f617a620240>
1,<omapy.oma.Recording object at 0x7f61b8b6f128>
2,<omapy.oma.Recording object at 0x7f617a6ad0b8>
3,<omapy.oma.Recording object at 0x7f617a6acdd8>
4,<omapy.oma.Recording object at 0x7f617a6af128>


In [10]:
sessions = []
for recording in recordings:
    recording_sessions = recording.get_annotation_sessions()
    sessions.append(recording_sessions.dictionary())

pd.DataFrame(sessions).head()

for session in sessions:
    print(pd.DataFrame(session).head())



       id                                              title
0  194060  Sibelius_Sym6_Beecham_HelsinkiPO_1954live_yout...
       id                                              title
0  195128  Sibelius_Sym6_Beecham_HelsinkiPO_1954live_yout...
       id                                           title
0  191657  Sibelius_Sym6_Ashkenazy_1984_ganz_fertig_I.csv
       id                                             title
0  193526  Sibelius_Sym6_JärviNeeme_2005_ganzt_fertig_I.csv
       id                                              title
0  189158  Upload of Sibelius_Sym6_Karajan_1967_ganzt_fer...
       id                                          title
0  189520  Sibelius_Sym6_Karajan_1980_ganzt_fertig_I.csv
       id                                              title
0  191390  Sibelius_Sym6_Schneevoigt_1934_ganzt_fertig_I.csv
       id                                         title
0  192725  Sibelius_Sym6_Vänskä_1997_ganzt_fertig_I.csv


In [ ]:
# take the id of the first session
my_session_id = sessions.dictionary()[0]["id"]

# provide the id of the desired session to fetch the annotation session
session = oma.annotation_session(my_session_id)
# let's store the dict representation of a session to a new variable session_data
session_data = session.dictionary()
# and take only the annotations (they describe the beats) form the session_data
annotations = session_data["annotations"]
# view the first 3 elements of annotations
annotations[0:3]

Example:
```
[{'barNumber': 1,
  'beatNumber': 1,
  'id': 187980,
  'momentOfPerception': 2.600634921},
 {'barNumber': 2,
  'beatNumber': 1,
  'id': 187981,
  'momentOfPerception': 5.026235828},
 {'barNumber': 3,
  'beatNumber': 1,
  'id': 187982,
  'momentOfPerception': 7.689795918}]
```
each annotation with given id describes the moment of perception of a given beat number at a given bar number

We are interested in the tempo.
The tempo per beat we define as 1/time per beat. Other definitions are possible, .e.g., beats per minute, but they are only proportionally different.

In [ ]:
# loading some libraries for working with numbers and data
import pandas as pd
import numpy as np

# pandas is nice for processing table-like data in a structure called data frame
# search the web for pandas
data_frame = pd.DataFrame(annotations)
# using the head() function to show the first couple of records
data_frame.head()

We want to know the duration of each bar, so we filter out eyerything that is not a "one beat".

In [ ]:
# this creates a true/false filter on the beatNumber
one_beats = data_frame['beatNumber'] == 1
# using this filter, we select only True rows from the data_frame
one_beats_only = data_frame[one_beats]
one_beats_only.head()

In [ ]:
# add another column to the data frame.
# how big is our data frame?
# take one column and use shape to get the number of row

column_shape = one_beats_only['id'].shape
# use this shape with np.zeros() to create a column containing zeros
# and apply this column to the data frame with title "barDuration"
one_beats_only['barDuration'] = np.zeros(column_shape)
one_beats_only.head()

In [ ]:
# iterate the table to calculate the barDuration. skip the last row
# because the last beat's duration cannot be calculated
for row_index in range(column_shape[0] - 1):
    #current_bar_start = one_beats_only["momentOfPerception"][row_index]
    current_bar_start = one_beats_only.iloc[row_index, 3]
    next_bar_start = one_beats_only.iloc[row_index + 1, 3]
    duration = next_bar_start - current_bar_start
    #write back result
    one_beats_only.iloc[row_index, 4] = duration

one_beats_only.head()

In [ ]:
# add another column for the inverse value.
# this time using some numpy features.
# search the web for numpy

# "slice" the column with the barDuration and convert it to a numpy array
durations = one_beats_only["barDuration"].to_numpy()
tempos = np.reciprocal(durations)
# add the tempos. np arrays can be added as columns
one_beats_only["barTempo"] = tempos
one_beats_only.head()

In [ ]:
# plot the tempo curve
# x and y axes can be numpy arrays too
x_axis = one_beats_only["barNumber"].to_numpy()
y_axis = one_beats_only["barTempo"].to_numpy()

# import plotting library
import matplotlib.pyplot as plt
# search the web for matplotlib
fig, ax = plt.subplots()
ax.set_xlabel("bar number")
ax.set_ylabel("tempo in bars per second")
ax.set_ylim([0,0.8])
ax.plot(x_axis, y_axis)